In [1]:
from minitorch.nn.parameter import Parameter

class BrokenModule:
    def __init__(self):
        self._parameters = {}
        self._modules = {}
    
    def parameters(self):
        params = list(self._parameters.values())
        for module in self._modules.values():
            params.extend(module.parameters())
        return params
    
    def zero_grad(self):
        for param in self.parameters():
            param.zero_grad()
    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)
    
    def forward(self, *args, **kwargs):
        raise NotImplementedError("Subclasses must implement the forward method.")
    
    def __repr__(self):
        rep = f"{self.__class__.__name__}(\n"
        for name, param in self._parameters.items():
            rep += f"  ({name}): {param},\n"
        for name, module in self._modules.items():
            rep += f"  ({name}): {module},\n"
        rep += ")"
        return rep

In [2]:
import random
from minitorch.engine import Tensor
from minitorch.nn.parameter import Parameter

class Linear(BrokenModule):
    def __init__(self,in_features, out_features, bias=True) -> None:
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        k= 1 / (in_features ** 0.5)
        
        weight_data = [[random.uniform(-k, k) for _ in range(out_features)] for _ in range(in_features)]
        self.weight = Parameter(Tensor(weight_data)) 

        if bias:
            bias_data = [random.uniform(-k, k) for _ in range(out_features)]
            self.bias = Parameter(Tensor(bias_data))
        else:
            self.bias = None

    
    def forward(self, x):
        if not isinstance(x, Tensor):
            raise TypeError("Input must be a Tensor.")
        if x.data.shape[1] != self.in_features:
            raise ValueError(f"Input shape {x.data.shape} does not match expected shape ({x.data.shape[0]}, {self.in_features}).")
        
        out = x @ self.weight
        if self.bias is not None:
            out += self.bias
        return out
    
    def __repr__(self):
        return f"Linear(in_features={self.in_features}, out_features={self.out_features}, bias={self.bias is not None})"

In [4]:
from minitorch.nn import Linear, Module
from minitorch import Tensor
from minitorch.nn.parameter import Parameter

class MiniTorchModel(BrokenModule):
    def __init__(self, in_feat, out_feat):
        super().__init__()
        self.layer1 = Linear(in_feat, 3)
        self.layer2 = Linear(3, out_feat)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

minitorch_model = MiniTorchModel(2, 1)
print(minitorch_model)

print(minitorch_model.layer1.weight)
print(minitorch_model.layer1.bias)

print(minitorch_model.layer2.weight)
print(minitorch_model.layer2.bias)



MiniTorchModel(
)
Parameter containing:
tensor([[ 0.5934  0.2652 -0.3113]
 [ 0.2123 -0.1907 -0.4582]], requires_grad=True)
Parameter containing:
tensor([ 0.5554 -0.1766  0.0723], requires_grad=True)
Parameter containing:
tensor([[-0.3387]
 [ 0.304 ]
 [ 0.4305]], requires_grad=True)
Parameter containing:
tensor([0.537], requires_grad=True)


In [5]:
print(minitorch_model.parameters())

print('-' * 50)

print(minitorch_model.parameters)
print(minitorch_model.zero_grad())

print(minitorch_model.layer1.weight.grad)

[]
--------------------------------------------------
<bound method BrokenModule.parameters of MiniTorchModel(
)>
None
[[0. 0. 0.]
 [0. 0. 0.]]
